In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -U --q transformers datasets accelerate sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:
%cd '/content/drive/MyDrive/NLU_NCKH/notebook/res_data_preprocesed'

/content/drive/.shortcut-targets-by-id/1btr8FZI1SJ1bZdaS-BqFfYlWxPaXfQOB/NLU_NCKH/notebook/res_data_preprocesed


# Import library

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import datasets
from sklearn.metrics import *

# Load data

In [5]:
df_train = pd.read_csv("train_res_preprocesed.csv")
df_dev = pd.read_csv("dev_res_preprocesed.csv")
df_test = pd.read_csv("test_res_preprocesed.csv")

print("Train: ", df_train.shape)
print("Dev: ",  df_dev.shape)
print("Test: ", df_test.shape )

Train:  (7028, 13)
Dev:  (771, 13)
Test:  (1938, 13)


# Preprocess data

In [6]:
# Defined aspect and polarity map
aspect_map = {
    "SERVICE#GENERAL": "Phục vụ",
    "RESTAURANT#GENERAL": "Nhà hàng nói chung",
    "RESTAURANT#PRICES": "Gía tiền nhà hàng",
    "RESTAURANT#MISCELLANEOUS": "Vấn đề khác",
    "FOOD#STYLE&OPTIONS": "Lựa chọn đồ ăn",
    "FOOD#QUALITY": "Chất lượng món ăn",
    "FOOD#PRICES": "Gía tiền món ăn",
    "DRINKS#QUALITY": "Chất lượng đồ uống",
    "DRINKS#PRICES": "Gía tiền đồ uống",
    "DRINKS#STYLE&OPTIONS": "Lựa chọn đồ uống",
    "LOCATION#GENERAL": "Địa điểm",
    "AMBIENCE#GENERAL": "Không gian",
}

polarity_map = {
    1: "tệ",
    2: "tạm",
    3: "tốt"
}

In [7]:
import numpy as np

def toSeq(row):
    dict_label = {1: "negative", 2: "neutral", 3: "positive"}
    seqs = []
    labels = [
        (aspect, polarity)
        for aspect, polarity in zip(row.index, row.values) if polarity != 0
    ]
    for label in labels:
        seq = aspect_map[label[0]] + " " + polarity_map[label[1]]
        seqs.append(seq)
    return ", ".join(seqs).capitalize()

# df_train.drop("review", axis=1).apply(toTextLabel, axis=1).iloc[0]
df_train["Labels"] = df_train.drop("review", axis=1).apply(toSeq, axis=1)
df_test["Labels"] = df_test.drop("review", axis=1).apply(toSeq, axis=1)
df_dev["Labels"] = df_dev.drop("review", axis=1).apply(toSeq, axis=1)
df_train['Labels'].head()

0    Gía tiền đồ uống tạm, lựa chọn đồ uống tạm
1                          Gía tiền nhà hàng tệ
2                          Chất lượng món ăn tệ
3                         Chất lượng món ăn tốt
4                         Gía tiền nhà hàng tốt
Name: Labels, dtype: object

In [8]:
label_lengths = df_train["Labels"].map(len)
review_lengths = df_train["review"].map(len)

# Tìm độ dài lớn nhất và chỉ số của hàng có độ dài lớn nhất
max_review_length = review_lengths.max()
max_review_index = review_lengths.idxmax()

max_labels_length = label_lengths.max()
max_labels_index = label_lengths.idxmax()

print(f"Max length review: {max_review_length}")
print(f"Index of max review: {max_review_index}")
print(f"Value of Review at max index: {df_train['Labels'][max_review_index]}")
print(f"Value of Review at max index: {df_train['review'][max_review_index]}")
print("-"*80)
print(f"Max length labels: {max_labels_length}")
print(f"Index of max length: {max_labels_index}")
print(f"Value of Labels at max index: {df_train['Labels'][max_labels_index]}")
print(f"Value of Labels at max index: {df_train['review'][max_labels_index]}")

Max length review: 432
Index of max review: 5475
Value of Review at max index: Chất lượng món ăn tốt, lựa chọn đồ ăn tốt, chất lượng đồ uống tốt, lựa chọn đồ uống tạm
Value of Review at max index: toast lava kim_sa món này cũng ngon không kém bánh mới vừa mang ra nóng_hổi hết sảy con bà bảy luôn mới vừa dùng dao cắt ra là sốt kim_sa tràn ra mà thèm chảy nước_miếng luôn á và 1 ly trà_xoài do uống li này một bữa ở bên đen đá pasteur thấy ngon quá nên từ đó thích món này luôn và bạn mình uống thấy cũng thích theo há há trà hoa_hồng ad thêm trân_châu trắng uống cũng được có kem béo là thành ngon hà roling_on_the_flor_laughing
--------------------------------------------------------------------------------
Max length labels: 105
Index of max length: 1136
Value of Labels at max index: Địa điểm tốt, nhà hàng nói chung tốt, chất lượng món ăn tốt, chất lượng đồ uống tốt, lựa chọn đồ uống tốt
Value of Labels at max index: quán trà sữa rất ngon nhiều đồ uống mới_lạ có các chỗ ngồi quay ra đường r

In [9]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

In [10]:
max_len_input = max(len(tokenizer.encode(review)) for review in df_train['review']) + 13
max_len_target = len(tokenizer.encode(df_train['Labels'][max_labels_index]))
print(max_len_input)
print(max_len_target)

140
28


In [11]:
import random
import torch
from torch.utils.data import Dataset, DataLoader


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [12]:
from datasets import Dataset
dataset_train =  Dataset.from_pandas(df_train)
dataset_dev = Dataset.from_pandas(df_dev)
dataset_test = Dataset.from_pandas(df_test)

example = dataset_train[0]

print("Review:", example["review"])
print("Traget:", example["Labels"])

Review: giá 53k cỡ vừa
Traget: Gía tiền đồ uống tạm, lựa chọn đồ uống tạm


In [13]:
dataset_train

Dataset({
    features: ['review', 'SERVICE#GENERAL', 'LOCATION#GENERAL', 'RESTAURANT#GENERAL', 'AMBIENCE#GENERAL', 'FOOD#PRICES', 'RESTAURANT#MISCELLANEOUS', 'DRINKS#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE&OPTIONS', 'RESTAURANT#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE&OPTIONS', 'Labels'],
    num_rows: 7028
})

# Prepare input for model

In [14]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import datasets

# Initialize constants and tokenizer
# MAX_LEN_INPUT = 170
# MAX_LEN_TARGET = 70
# MODEL_NAME = "VietAI/vit5-base"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_examples(examples):
    prefix = "aspect-based sentiment analysis: "
    reviews = [prefix + review for review in examples['review']]
    labels = [label for label in examples['Labels']]

    # Tokenize the reviews
    model_inputs = tokenizer(
        reviews,
        max_length=max_len_input,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Tokenize the labels
    labels = tokenizer(
        labels,
        max_length=max_len_target,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

    # Adjust labels for compatibility with PyTorch
    temp_labels = labels['input_ids']
    temp_labels[temp_labels == tokenizer.pad_token_id] = -100

    # Add target_ids and target_mask to model_inputs
    model_inputs['labels'] = temp_labels
    model_inputs['decoder_attention_mask'] = labels['attention_mask']

    return model_inputs


# Apply the preprocessing function to the datasets
dataset_train = dataset_train.map(preprocess_examples, batched=True)
dataset_dev = dataset_dev.map(preprocess_examples, batched=True)
dataset_test = dataset_test.map(preprocess_examples, batched=True)

# Set the format of the datasets to PyTorch tensors
dataset_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_dev.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])
dataset_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'])

Map:   0%|          | 0/7028 [00:00<?, ? examples/s]

Map:   0%|          | 0/771 [00:00<?, ? examples/s]

Map:   0%|          | 0/1938 [00:00<?, ? examples/s]

# Define metrics

In [15]:
import re
import numpy as np
from sklearn.metrics import classification_report

true = ["Địa điểm tốt, nhà hàng nói chung tốt, chất lượng món ăn tốt", "Chất lượng đồ uống tốt, lựa chọn đồ uống tốt"]
pred = ["Gía tiền món ăn tốt, nhà hàng nói chung tốt, chất lượng món ăn tạm", "Chất lượng đồ uống tệ, lựa chọn đồ uống tốt"]

polarity_map = {
    "tệ": "negative",
    "tạm": "neutral",
    "tốt": "positive"
}

aspects = df_train.drop(["review", "Labels"], axis=1).columns.tolist()
aspects = [aspect.lower() for aspect in aspects]
sentiments = {'negative': 1, 'neutral': 2, 'positive': 3}
replacements = {0: 'none', 1: 'negative', 2: 'neutral', 3: 'positive'}

def parse_output(batchs):
    outputs = []
    for label in batchs:
      output = []
      sentences = label.split(',')
      sentences = [sentence.strip().lower() for sentence in sentences]
      for sentence in sentences:
          aspect, polarity = None, None
          for aspect_key, aspect_value in aspect_map.items():
              if aspect_value.lower() in sentence:
                aspect = aspect_key
                break
          for polarity_key, polarity_value in polarity_map.items():
              if polarity_key in sentence:
                polarity = polarity_value
                break
          output.append((aspect, polarity))

      outputs.append(output)
    aspects_polarity = []
    for output in outputs:
        y = [0] * len(aspects)
        for aspect, polarity in output:
            index = aspects.index(aspect.lower())
            y[index] = sentiments[polarity]
        aspects_polarity.append(y)
    return aspects_polarity
# y_true = parse_output(true)
# y_pred = parse_output(pred)

In [16]:
def aspect_detection_eval(y_true, y_pred):
    aspect_test = []
    aspect_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_test.append(bool(col_test) * aspects[index])
            aspect_pred.append(bool(col_pred) * aspects[index])
    metrics = classification_report(aspect_test, aspect_pred, zero_division=1, digits=4)
    return metrics

def sentiment_classification_eval(y_true, y_pred):
    y_true_flat = np.array(y_true).flatten()
    y_pred_flat = np.array(y_pred).flatten()
    target_names = list(map(str, replacements.values()))
    metrics = classification_report(y_true_flat, y_pred_flat, zero_division=1, target_names=target_names, digits=4)
    return metrics

def combination_eval(y_true, y_pred):
    aspect_polarity_true = []
    aspect_polarity_pred = []

    for row_test, row_pred in zip(y_true, y_pred):
        for index, (col_test, col_pred) in enumerate(zip(row_test, row_pred)):
            aspect_polarity_true.append(f'{aspects[index]},{replacements[col_test]}')
            aspect_polarity_pred.append(f'{aspects[index]},{replacements[col_pred]}')
    metrics = classification_report(aspect_polarity_true, aspect_polarity_pred, zero_division=1, digits=4)
    return metrics

# print(list(map(str, replacements.values())))
# # Đánh giá
# print("## Aspect Detection Evaluate ##")
# metrics = aspect_detection_eval(y_true, y_pred)
# print(metrics)

# print("\n## Sentiment Classification Evaluate ##")
# metrics = sentiment_classification_eval(y_true, y_pred)
# print(metrics)

# print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
# metrics = combination_eval(y_true, y_pred)
# print(metrics)

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Device:', device)
!nvidia-smi

Device: cuda
Mon Jun 17 14:58:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+------------

# Training model

In [ ]:
import wandb
api = wandb.Api()
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 21130500 (quid). Use `wandb login --relogin` to force relogin


True

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
# model.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir= f'./seqtoseq_unified_{MODEL_NAME.replace("/", "_")}',
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    warmup_steps=500,
    eval_strategy="epoch",
    save_strategy='no'
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_dev,
    # compute_metrics=compute_metrics
)

# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,No log,0.356235
2,2.363300,0.242341


Epoch,Training Loss,Validation Loss
1,No log,0.356235
2,2.363300,0.242341
3,0.282200,0.214656
4,0.186300,0.221432


TrainOutput(global_step=1760, training_loss=0.8270477078177713, metrics={'train_runtime': 1138.7499, 'train_samples_per_second': 24.687, 'train_steps_per_second': 1.546, 'total_flos': 4680983184998400.0, 'train_loss': 0.8270477078177713, 'epoch': 4.0})

# Load model

In [18]:
save = "/content/drive/MyDrive/NLU_NCKH/notebook/model/seqtoseq_unified"
# trainer.save_model(save)

In [ ]:
import gc
del model, tokenizer,
torch.cuda.empty_cache()
gc.collect()

0

In [19]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
device = "cuda" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(save).to(device)

# Prepare input to inference

In [20]:
class customDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.labels)

# Inference and Evaluate

In [22]:
from torch.utils.data import DataLoader
from tqdm import tqdm
# Evaluate the model
# results = trainer.evaluate()
# Generate predictions
# MAX_LEN_INPUT = 170
# MAX_LEN_TARGET = 70

def generate_predictions(dataset):
    prefix = "aspect-based sentiment analysis: "
    input_texts = [prefix + review for review in dataset['review']]
    y_test = dataset['Labels']
    # tokenizer.add_tokens(["#", "&"])
    # Tokenize the reviews
    inputs = tokenizer(
        input_texts,
        max_length=max_len_input,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    encoding_data = customDataset(inputs, y_test)
    data_loader = DataLoader(encoding_data, batch_size=64, shuffle=False, num_workers=0)
    y_preds = []

    model.eval()
    for index, data in enumerate(tqdm(data_loader, desc="Predicting")):
      outputs = model.generate(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          max_length=max_len_target,
          num_beams=4,
          early_stopping=True
      )
      generate = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
      y_preds.extend(generate)

    return y_test, y_preds

dataset_test = datasets.Dataset.from_pandas(df_test)

y_test, y_preds = generate_predictions(dataset_test)

Predicting: 100%|██████████| 31/31 [01:33<00:00,  3.03s/it]


In [23]:
for y, p in zip(y_test[:10], y_preds[:10]):
  print(y)
  print(p)
  print("-"*80)

Vấn đề khác tốt
Vấn đề khác tốt
--------------------------------------------------------------------------------
Chất lượng đồ uống tạm
Chất lượng đồ uống tạm
--------------------------------------------------------------------------------
Phục vụ tệ
Phục vụ tệ
--------------------------------------------------------------------------------
Nhà hàng nói chung tốt, không gian tệ
Nhà hàng nói chung tạm
--------------------------------------------------------------------------------
Phục vụ tệ
Phục vụ tệ
--------------------------------------------------------------------------------
Chất lượng món ăn tốt, lựa chọn đồ ăn tốt
Chất lượng món ăn tốt, lựa chọn đồ ăn tốt
--------------------------------------------------------------------------------
Chất lượng món ăn tốt
Chất lượng món ăn tốt
--------------------------------------------------------------------------------
Vấn đề khác tệ
Vấn đề khác tạm
--------------------------------------------------------------------------------
Gía tiền đ

In [24]:
y_true = parse_output(y_test)
y_pred = parse_output(y_preds)

print("## Aspect Detection Evaluate ##")
report = aspect_detection_eval(y_true, y_pred)
print(report)

print("\n## Sentiment Classification Evaluate ##")
report = sentiment_classification_eval(y_true, y_pred)
print(report)

print("\n## Combination Evaluate (Aspect + Polarity detection) ##")
report = combination_eval(y_true, y_pred)
print(report)

## Aspect Detection Evaluate ##
                          precision    recall  f1-score   support

                             0.9797    0.9799    0.9798     20627
        ambience#general     0.8673    0.8634    0.8653       227
           drinks#prices     0.8605    0.7872    0.8222        47
          drinks#quality     0.8551    0.8719    0.8634       203
    drinks#style&options     0.7462    0.7519    0.7490       129
             food#prices     0.8305    0.8750    0.8522       112
            food#quality     0.8780    0.9097    0.8936       554
      food#style&options     0.7852    0.8284    0.8062       437
        location#general     0.9208    0.8942    0.9073       104
      restaurant#general     0.7964    0.7012    0.7458       251
restaurant#miscellaneous     0.6947    0.6276    0.6594       145
       restaurant#prices     0.8952    0.8034    0.8468       117
         service#general     0.9283    0.9406    0.9344       303

                accuracy                  

In [27]:
save = pd.DataFrame({
    'y_test': y_true,
    'y_pred': y_pred
})

save.to_csv("Unified_Seq2Seq_predicted.csv")